In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jaxlie import SE3, SO3
import cyipopt

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *

In [2]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
panda = Robot(world.vis, "panda", panda_model, alpha=0.5)
panda.reduce_dim([7, 8], [0.04, 0.04])

In [4]:
# ik
robot_dim = 7
horizon = 5
dim = robot_dim * horizon
dt = 0.1

to_mat = lambda x: x.reshape(-1, robot_dim)
to_vec = lambda x: x.flatten()

def to_qdots_mat(q0, qs):
    qs = jnp.vstack([q0, to_mat(qs)])
    return (qs[1:] - qs[:-1]) /dt

In [5]:
# Kinematics
def skew(v):
    v1, v2, v3 = v
    return jnp.array([[0, -v3, v2],
                      [v3, 0., -v1],
                      [-v2, v1, 0.]])

def get_rotvec_angvel_map(v):
    vmag = jnp.linalg.norm(v)
    vskew = skew(v)
    return jnp.eye(3) \
        - 1/2*skew(v) \
        + vskew@vskew * 1/vmag**2 * (1-vmag/2 * jnp.sin(vmag)/(1-jnp.cos(vmag)))

@jax.jit
def get_ee_fk_jac(q):
    fks = panda_model.fk_fn(q)
    p_ee = fks[-1][-3:]
    rotvec_ee = SO3(fks[-1][:4]).log()
    E = get_rotvec_angvel_map(rotvec_ee)
    jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        jac.append(jnp.hstack([lin_vel, rot_axis]))
    jac = jnp.array(jac).T
    jac = jac.at[3:, :].set(E @ jac[3:, :])
    return jnp.hstack([p_ee, rotvec_ee]), jac

In [6]:
frame = Frame(world.vis, "frame")
frame_ee = Frame(world.vis, "frame_ee")

In [14]:
def make_pose():
    return SE3.from_rotation_and_translation(
        SO3(np.random.random(4)).normalize(),
        np.random.uniform([-0.3,-0.5,0.3],[0.6, 0.5, 0.8])
    )

In [15]:
import time
W = np.diag([1,1,1,0.3,0.3,0.3])
def to_posevec(pose:SE3):
    return jnp.hstack([
        pose.translation(), pose.rotation().log()
    ])
def to_SE3(posevec:Array):
    return SE3.from_rotation_and_translation(
        SO3.exp(posevec[3:]), posevec[:3]
    )

def pose_error(q, target:SE3):
    ee_posevec, _ = get_ee_fk_jac(q)
    target_posevec = to_posevec(target)
    err = (target_posevec - ee_posevec)
    val = err@W@err#jnp.sum(err[:3]**2) + 0.3*jnp.sum(err[3:]**2)
    return val

def pose_error_grad(q, target:SE3):
    ee_posevec, jac = get_ee_fk_jac(q)
    target_posevec = to_posevec(target)
    err = (target_posevec - ee_posevec)
    grad_err = - 2*jac.T @ W @err
    return grad_err

def pose_error_hess(q, lagrange, obj_factor):
    _, jac = get_ee_fk_jac(q)
    return jac.T@W@jac

In [38]:
x = jnp.zeros(dim)

In [39]:
get_ee_point = lambda q:panda_model.fk_fn(q)[-1][-3:]
get_ee_points = jax.vmap(get_ee_point)
points = get_ee_points(to_mat(x))
traj = DottedLine(world.vis, "traj", points)

In [70]:
def objective(x, target:SE3):
    val = jax.vmap(pose_error, in_axes=(0,None))(to_mat(x), target).sum()
    traj.reload(points=get_ee_points(to_mat(x)))
    return val

def gradient(x, target:SE3):
    grads = jax.vmap(pose_error_grad, in_axes=(0,None))(to_mat(x), target)
    return grads.flatten()

def hessian(x, lagrange, obj_factor):
    hessians = jax.vmap(pose_error_hess, in_axes=(0, None, None))(to_mat(x), 0,1.)
    result = jnp.zeros([dim, dim])
    for i in range(5):
        start, end = i*robot_dim, (i+1)*robot_dim
        result = result.at[start:end, start:end].set(hessians[i])
    return result

qvel_lb = -jnp.ones(dim)
qvel_ub = jnp.ones(dim)
def constraints(x, q0):
    qdots = to_qdots_mat(q0, x).flatten()
    ub_viol = qvel_ub - qdots
    lb_viol = qdots - qvel_lb
    return jnp.hstack([lb_viol, ub_viol])
jacobian = jax.jacrev(constraints)


In [74]:
pose_d = make_pose()
frame.set_pose(pose_d)

In [44]:
#state initialize
x = jnp.tile(panda.neutral, horizon)

In [21]:
import cyipopt
from functools import partial

In [82]:
class Prob:
    pass
prob = Prob()
setattr(prob, "objective", partial(objective, target=pose_d))
setattr(prob, "gradient", partial(gradient, target=pose_d))
setattr(prob, "constraints", partial(constraints, q0=panda.neutral))
setattr(prob, "jacobian", partial(jacobian, q0=panda.neutral))
#setattr(prob, "hessian", hessian)

ipopt = cyipopt.Problem(
    n=35, m=dim*2,
    problem_obj=prob,
    lb=np.tile(panda.lb,horizon), ub=np.tile(panda.ub, horizon),
    cl=np.zeros(dim*2), cu=np.full(dim*2,np.inf)
)
ipopt.add_option("nlp_scaling_method", "none")
ipopt.add_option("mu_strategy", "adaptive")
ipopt.add_option("acceptable_tol", 0.01)
#ipopt.add_option("acceptable_obj_change_tol", 0.001)
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("max_iter", 100)
xsol, info = ipopt.solve(np.tile(panda.neutral, horizon))

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2450
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       35
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       35
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       70
        inequality constraints with only lower bounds:       70
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.1271391e+00 0.00e+00 2.74e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [81]:
panda.set_joint_angles(to_mat(xsol)[4])

In [ ]:
traj.reload(points=)

In [1689]:
def rot_error(q, rot_d):
    ee_pose, _ = get_ee_fk_jac(q)
    orn_diff = SE3(ee_pose).rotation().inverse() @ rot_d
    orn_error = orn_diff.parameters()[1:]
    return 1/2 * orn_error @ orn_error

In [1680]:
def get_qtn_map(qtn):
    q0, q1, q2, q3 = qtn
    return jnp.array([
        [-q1, q0, -q3, q2],
        [-q2, q3, q0, -q1],
        [-q3, -q2, q1, q0],
    ])
def skew(v):
    v1, v2, v3 = v
    return jnp.array([[0, -v3, v2],
                      [v3, 0., -v1],
                      [-v2, v1, 0.]])

In [1667]:
qtn = make_pose().rotation().parameters()
rot_d = make_pose().rotation()

In [1690]:
jax.grad(rot_error)(panda.neutral, rot_d)

Array([-0.18696576, -0.02868757, -0.18696575,  0.02868757,  0.02062087,
        0.02868757,  0.18482518], dtype=float32)

In [1707]:
q = panda.neutral

In [1708]:
ee_pose, jac = get_ee_fk_jac(q)
orn_diff = SO3(qtn).inverse() @ rot_d
orn_error = orn_diff.parameters()[1:]

In [1711]:
qtn_d = rot_d.parameters()
qval, qvec = qtn_d[0], qtn_d[1:]
H = get_qtn_map(ee_pose[:4]) #ee_pose[:4]

In [1715]:
1/2*H.T@jac[3:,:]

Array([[-0.0361715 , -0.36486605, -0.0361715 ,  0.36486605, -0.33988822,
         0.36486608, -0.06478072],
       [-0.36486605,  0.0361715 , -0.36486605, -0.03617152,  0.00635148,
        -0.03617148,  0.35078028],
       [ 0.33988836,  0.00635018,  0.33988836, -0.00635016, -0.03617271,
        -0.00635021, -0.3356127 ],
       [ 0.00635018, -0.33988836,  0.00635018,  0.33988836,  0.364866  ,
         0.3398884 ,  0.10060166]], dtype=float32)

In [1712]:
1/2 * orn_error @ jnp.vstack([qvec, -jnp.eye(3)*qval - skew(qvec)]).T @ H.T @ jac[3:,:]

Array([-0.04278407, -0.44524294, -0.04278407,  0.44524294, -0.1600342 ,
        0.445243  , -0.0058739 ], dtype=float32)

In [1705]:
jac.shape

(7, 7)

In [1240]:
def pose_error(pose:Array, pose_d:SE3, weight=0.3):
    pos_error = pose_d.translation() - pose[-3:]
    orn_diff = SO3(pose[:4]).inverse() @ pose_d.rotation()
    orn_error = orn_diff.parameters()[1:]
    return pos_error @ pos_error + weight * orn_error @ orn_error

In [1297]:
def pose_error(q, pose_d):
    ee_pose = panda_model.fk_fn(q)[-1]
    pos_error = pose_d.translation() - ee_pose[-3:]
    orn_diff = SO3(ee_pose[:4]).inverse() @ pose_d.rotation()
    orn_error = orn_diff.parameters()[1:]
    return pos_error @ pos_error + orn_error @ orn_error

In [1659]:
q = panda.neutral

In [1660]:
q_grad = jax.grad(pose_error)(q, pose_d)

In [1661]:
q_grad

Array([-0.11435245,  0.27144504, -0.11435244, -0.20746024,  0.02325282,
       -0.33237973,  0.1296407 ], dtype=float32)

In [1658]:
q_grad = jax.grad(pose_error)(q, pose_d)
q -= q_grad*0.5
panda.set_joint_angles(q)
#print(pose_error(ee_pose, pose_d))

In [1296]:
ee_pose, jac = get_ee_fk_jac(q)
q_grad = jax.grad(pose_error)(ee_pose, pose_d)[-3:] @ jac[:3,:]
q -= q_grad*1.
panda.set_joint_angles(q)
print(pose_error(ee_pose, pose_d))

4.7153628e-07


In [1024]:
jax.grad(pose_error)(ee_pose, pose_d)

Array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.841351  ,
       -0.608469  , -0.07974565], dtype=float32)

In [292]:
ee_pose, jac = get_ee_fk_jac(q)
jax.grad(pos_error)(ee_pose[-3:], pose_d.translation()) @ jac[:3, :]

Array([1.50614709e-01, 9.87606347e-02, 3.19140136e-01, 3.22840922e-02,
       1.12083755e-01, 1.27956733e-01, 3.96228828e-09], dtype=float32)

In [285]:
def pose_error(q, pose_d):
    ee_pose, jac = get_ee_fk_jac(q)
    pos_error = pose_d.translation() - ee_pose[-3:]
    orn_diff = SO3(ee_pose[:4]).inverse() @ pose_d.rotation()
    orn_error = orn_diff.parameters()[1:]
    return pos_error @ pos_error + orn_error @ orn_error

def pose_error_grad(q, pose_d):
    ee_pose, jac = get_ee_fk_jac(q)
    pos_error = pose_d.translation() - ee_pose[-3:]
    orn_diff = SO3(ee_pose[:4]).inverse() @ pose_d.rotation()
    orn_error = orn_diff.parameters()[1:]
    error = jnp.hstack([pos_error, orn_error])
    return - error @ jac[:3, :]

def get_qtn_angvel_map(qtn):
    q0, q1, q2, q3 = qtn
    return 1/2 * jnp.array([
        [-q1, q0, -q3, q2],
        [-q2, q3, q0, -q1],
        [-q3, -q2, q1, q0],
    ]).T
def qtn_error(qtn:Array, qtn_d:Array):
    orn_diff = SO3(qtn).inverse() @ SO3(qtn_d)
    orn_error = orn_diff.parameters()[1:]
    return orn_error@orn_error

In [98]:
q = panda.neutral

In [244]:
ee_pose, jac = get_ee_fk_jac(q)

In [260]:
ee_pose, jac = get_ee_fk_jac(q)
pos_error = pose_d.translation() - ee_pose[-3:]
orn_diff = SO3(ee_pose[:4]).inverse() @ pose_d.rotation()
orn_error = orn_diff.parameters()[1:]
error = jnp.hstack([pos_error, orn_error])

In [266]:
2*orn_error @ 

Array([ 0.03801263, -0.41752088,  0.6661166 ], dtype=float32)

In [269]:
orn_d = pose_d.rotation().parameters()

In [284]:
jax.grad(qtn_error)(ee_pose[:4], pose_d.parameters()[:4])

Array([-0.07927534, -0.44725028,  0.55431426,  0.3254203 ], dtype=float32)

In [273]:
jnp.hstack([qtn_d[1:,None], -jnp.eye(3)*qtn_d])

Array([[0.81050926],
       [0.5543198 ],
       [0.18218307]], dtype=float32)

In [258]:
orn_error @ get_qtn_angvel_map(ee_pose[:4])[1:] @ jac[3:]

Array([-0.06068315, -0.15786007, -0.10256918,  0.16130203, -0.05043432,
        0.17560862,  0.06313524], dtype=float32)

In [254]:
orn_error

In [238]:
q_grad = pos_error_grad(q, pose_d)
q -= q_grad*0.1
panda.set_joint_angles(q)

Array(0.2659558, dtype=float32)

In [789]:
SO3(ee_pose[:4])

SO3(wxyz=[0.61263996 0.         0.55449    2.77159   ])

In [783]:
jac[3:,:]

Array([[-0.45957762,  0.52428114, -0.45957762, -0.52428114,  0.85152006,
        -0.52428114,  0.68845195],
       [ 1.4331958 ,  0.21716428,  1.4331958 , -0.21716419,  0.11006793,
        -0.21716441, -1.3383926 ],
       [ 0.10288757, -1.338393  ,  0.10288757,  1.338393  ,  0.68845165,
         1.3383931 ,  0.10288784]], dtype=float32)

In [275]:
def to_posevec(pose:SE3):
    return jnp.hstack([
        pose.translation(), pose.rotation().log()
    ])
def to_SE3(posevec:Array):
    return SE3.from_rotation_and_translation(
        SO3.exp(posevec[3:]), posevec[:3]
    )

@jax.jit
def pose_err(q, target:SE3):
    ee_posevec, _ = get_ee_fk_jac(q)
    target_posevec = to_posevec(target)
    err = (target_posevec - ee_posevec)
    val = jnp.sum(err**2)
    return val

@jax.jit
def pose_err_grad(q, target:SE3):
    ee_posevec, jac = get_ee_fk_jac(q)
    target_posevec = to_posevec(target)
    err = (target_posevec - ee_posevec)
    grad_err = - 2*jac.T@err
    return grad_err

In [711]:
q = panda.neutral

In [774]:
q_grad = pose_err_grad(q, target)
q -= q_grad*0.1
panda.set_joint_angles(q)
print(pose_err(q, target))

0.209952


In [9]:
def obj_fn(x, target):
    return pose_err(x, target)
def grad_fn(x, target):
    return pose_err_grad(x, target)

In [38]:
def make_random_pose():
    return SE3.from_rotation_and_translation(
        SO3(np.random.random(4)).normalize(),
        np.random.uniform([-0.,-0.5,0.3],[0.5,0.5,0.8])
    )

In [ ]:
frame = Frame(world.vis, "frame")

In [55]:
target = make_random_pose()
frame.set_pose(target)

In [56]:
from functools import partial
target = make_random_pose()
objective = partial(obj_fn, target=target)
gradient = partial(grad_fn, target=target)
# constraints = partial(constr_fn, q0=q0)
# jacobian = partial(constr_fn_jac, q0=q0)
class Prob:
    pass
prob = Prob()
setattr(prob, "objective", objective)
setattr(prob, "gradient", gradient)
# setattr(prob, "constraints", constraints)
# setattr(prob, "jacobian", jacobian)

ipopt = cyipopt.Problem(
    n=7, m=0,
    problem_obj=prob,
    lb=panda_model.lb[:7], ub=panda_model.ub[:7],
    #cl=np.zeros(dim), cu=np.full(dim, np.inf)
)

In [59]:
#ipopt.add_option("acceptable_obj_change_tol", 1.)
ipopt.add_option("acceptable_tol", 0.0001)
ipopt.add_option("acceptable_iter", 2)
ipopt.add_option("print_level", 5)
ipopt.add_option("max_iter", 100)

xsol, info = ipopt.solve(panda.neutral)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.6770287e+00 0.00e+00 5.14e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [61]:
panda.set_joint_angles(xsol)